# MITCH DB Loader

To make it easier to handle the experiment data from the MITCH, the .txt files will be loaded in a database named Bootsensing based on sql. You can access all data directly from the there with commands defined in this notebook. In addition we will add information about the experiment, the boot and the user.

In [47]:
import pandas as pd
import numpy as np
import sqlite3
import csv

Within filename you have to select the files to be loaded. It is important to define the experiment, the boot and the user for each file with the appropiate variable. In df_name please define the name of the file and in tablename the name of the table in the database. 

In [72]:
filename = ['13.01.2021 165241', '13.01.2021 183219', '13.01.2021 165315','13.01.2021 183301']
#TimeShift = [0, 0] # Timeshift to match with other data [ms]
experiment = ['Uphill','Downhill','Uphill','Downhill']
bootname = ['Speedfit-SX', 'Speedfit-SX','Speedfit-DX', 'Speedfit-DX']
username = ['Marilena', 'Marilena','Marilena', 'Marilena']
df_name = 'Kuhleiten'
tablename = 'Field_05exp'

In this section all Mitch files are put togheter in a common data file with the same column names as in the original Mitch file.

In [73]:
# Create one list to append all files
appendList = []
# Define columns names of the list (same as in the files)
col_names = ['Time','AccX','AccY','AccZ','GyroX','GyroY','GyroZ','MagnX','MagnY','MagnZ','P0','P1','P2','P3','P4','P5','P6','P7','Temperature']

for i in range(len(filename)):
    # Read txt files and add column names
    ReadFile = pd.read_csv ('Files/'+ filename[i] + '.txt', delimiter='\t', skiprows=9)
    ReadFile.columns=col_names
    # Create timestamp with date and time
    ReadFile.Time = pd.to_datetime(ReadFile.Time, unit='ms')
    
    # Append each ReadFile to the appendList
    appendList.append(ReadFile)

# Transform appendList in to pandas format
DataFile = pd.concat(appendList)

All data is exported in one single csv file with the name you defined at the beginning in df_name.

In [74]:
data = DataFile.to_csv (df_name + '.csv', index=None)

To store all information about the experiments, a dataframe called infotable is created. In this separate table the timestamp of the experiment file, the experiment description, the name of the boot and the user will be stored in the database. Each time new data is added, this table needs to be updated too.

In [75]:
# Create infotable DataFrame with experiments infos
infotable = pd.DataFrame() # initialize pd DataFrame infotable
starttime = []
endtime = []

for i in range(len(filename)): 
    exp_starttime = appendList[i]['Time'].iloc[0] # select full starttime timestamp for starting time of each experiment
    exp_endtime = appendList[i]['Time'].iloc[-1]
    starttime.append(exp_starttime) 
    endtime.append(exp_endtime) 
    
# Define infotable columns    
infotable['Starttime'] = starttime
infotable['Endtime'] = endtime
infotable['Experiment'] = experiment
infotable['Boot'] = bootname
infotable['User'] = username


First, you have to connect with the database. Within the database a table with the name defined in df_name for each experiment will be created and filled with the above loaded data. Once the data is stored, the connection to the database need to be closed.

In [76]:
# Connect with database and create curser
con = sqlite3.connect('BootsensingDB.db')
cur = con.cursor()

# Create and access infotable
cur.execute("""CREATE TABLE IF NOT EXISTS {}(Starttime TEXT, Endtime TEXT, Experiment TEXT,Boot TEXT,User TEXT)""".format('infotable'))
# Commit the changes
con.commit()
# Append data to infotable
infotable.to_sql('infotable', con, if_exists='append', index=False)

# Create a new table for data
# Types in SQLite: str -> TEXT , int -> INTEGER , float -> REAL
cur.execute("""CREATE TABLE IF NOT EXISTS {}(Time TEXT,AccX REAL, AccY REAL,
                                             AccZ REAL, GyroX REAL, GyroY REAL,GyroZ REAL, MagnX REAL, MagnY REAL,
                                             MagnZ REAL, P0 REAL,P1 REAL, P2 REAL, P3 REAL, P4 REAL, P5 REAL, P6 REAL,
                                             P7 REAL, Temperature REAL)""".format(tablename))
# Commit the changes
con.commit()
# Load data into DataFrame
#filename = tablename + '.csv'
df = pd.read_csv(df_name + '.csv', delimiter=',')
# Load DataFrame in database  
df.to_sql(tablename, con, if_exists='replace', index=False)
# Close the database connection
con.close()

# Infotable query

To get the experiment information from the infotable a query using the timestamp of the starting time of the experiment needs to be done.

In [28]:
# Enter date and time of the dataline to search infos
date = '2021-01-08' # yyyy - mm - dd format
time = '13:48:30.5000' # hh:mm:ss.uuuuuu

# Create timestamp for query in right format
timestamp_query = date + " " + time

# Database connection and cursor
con = sqlite3.connect('BootsensingDB.db')
cur = con.cursor()

cur.execute("SELECT Experiment, Boot, User FROM infotable WHERE (Starttime <= ? AND ? <= Endtime)", (timestamp_query,timestamp_query,))

myresult = cur.fetchall()

con.close()

print(myresult)

[('DXcold', '1-DX', 'User 1')]
